In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
master_df = pd.read_csv('master_data_table.csv')
feed_df = pd.read_csv('./Example Inputs/feed9.csv')

df = master_df.merge(feed_df, how='outer', on='entry').dropna()

df.head()

,Unnamed: 0,wals code,name,value,description_x,latitude,longitude,area,title,entry,description_y
0,0.0,kiw,Kiwai (Southern),1.0,Small,-8.000000,143.500000,Phonology,Consonant Inventories,Consonant Inventories 1,Small
1,6.0,ach,Aché,1.0,Small,-25.250000,-55.166667,Phonology,Consonant Inventories,Consonant Inventories 1,Small
2,12.0,ain,Ainu,1.0,Small,43.000000,143.000000,Phonology,Consonant Inventories,Consonant Inventories 1,Small
3,15.0,akw,Akawaio,1.0,Small,6.000000,-59.500000,Phonology,Consonant Inventories,Consonant Inventories 1,Small
4,16.0,abm,Alabama,1.0,Small,32.333333,-87.416667,Phonology,Consonant Inventories,Consonant Inventories 1,Small


In [3]:
# # location = list(zip(df.latitude, df.longitude))
# df['latitude'] = latitude
df['entry'] = df['entry'].str.replace('/', ' or ')
df['entry'] = df['entry'].str.replace(' ', '_')

In [4]:
df

,Unnamed: 0,wals code,name,value,description_x,latitude,longitude,area,title,entry,description_y
0,0.0,kiw,Kiwai (Southern),1.0,Small,-8.000000,143.500000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
1,6.0,ach,Aché,1.0,Small,-25.250000,-55.166667,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
2,12.0,ain,Ainu,1.0,Small,43.000000,143.000000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
3,15.0,akw,Akawaio,1.0,Small,6.000000,-59.500000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
4,16.0,abm,Alabama,1.0,Small,32.333333,-87.416667,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
...,...,...,...,...,...,...,...,...,...,...,...
84,536.0,yag,Yagua,1.0,Small,-3.500000,-72.000000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
85,542.0,yar,Yareba,1.0,Small,-9.500000,148.500000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
86,543.0,yaw,Yawa,1.0,Small,-1.750000,136.250000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small
87,548.0,yid,Yidiny,1.0,Small,-17.000000,145.750000,Phonology,Consonant Inventories,Consonant_Inventories_1,Small


In [5]:
# entry = df['entry']
# dummies_entry = pd.get_dummies(entry)
# df['dummies_entry'] = dummies_entry
# df

In [6]:
X = df['entry']
y = df['latitude']

X.shape, y.shape

((88,), (88,))

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(X)
encoded_X = label_encoder.transform(X)

In [8]:
for label, original_class in zip(encoded_X, X):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Cons

In [9]:
one_hot_X = to_categorical(encoded_X)
# cols = [1,2,3,4,5,6,7,8,9,10]
# df = df[df.columns[cols]]
# df
one_hot_X[0:3,0:5]

array([[1.],
       [1.],
       [1.]], dtype=float32)

In [10]:
# # Step 1: Label-encode data set
# label_encoder = LabelEncoder()
# label_encoder.fit(y)
# encoded_y = label_encoder.transform(y)

In [11]:
# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, random_state=1)

In [12]:
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
classifier.fit(one_hot_X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = classifier.predict(one_hot_X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 271.83296702418403
R-squared (R2 ): -2.220446049250313e-16


In [15]:
predicted_lat = classifier.predict(one_hot_X)

In [16]:
predicted_lat_avg = np.average(predicted_lat)
print(predicted_lat_avg)

-0.16628768


In [17]:
X = df['entry']
y = df['longitude']

X.shape, y.shape

((88,), (88,))

In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(X)
encoded_X = label_encoder.transform(X)

In [19]:
for label, original_class in zip(encoded_X, X):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Consonant_Inventories_1
Encoded Label: 0
------------
Original Class: Cons

In [20]:
one_hot_X = to_categorical(encoded_X)
one_hot_X[0:3,0:5]

array([[1.],
       [1.],
       [1.]], dtype=float32)

In [21]:
# create training and testing data
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, random_state=1)

In [22]:
classifier = LinearRegression()
classifier

classifier.fit(one_hot_X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
# Use our model to predict a value
predicted = classifier.predict(one_hot_X)

# Score the prediction with mse and r2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

predicted_lon = classifier.predict(one_hot_X)

predicted_lon_avg = np.average(predicted_lon)
print(predicted_lon_avg)

Mean Squared Error (MSE): 11040.0888734204
R-squared (R2 ): -6.661338147750939e-16
55.513634


In [24]:
print(str(predicted_lat_avg) + ', ' + str(predicted_lon_avg))

-0.16628768, 55.513634


In [25]:
location_df = pd.DataFrame()
location_df['Latitude'] = predicted_lat
location_df['Longitude'] = predicted_lon
location_df

,Latitude,Longitude
0,-0.166288,55.513634
1,-0.166288,55.513634
2,-0.166288,55.513634
3,-0.166288,55.513634
4,-0.166288,55.513634
...,...,...
83,-0.166288,55.513634
84,-0.166288,55.513634
85,-0.166288,55.513634
86,-0.166288,55.513634


In [26]:
location_df.to_csv('location_list.csv')